# JupyterHub Notebook

### This notebook server is hosted on the OpenShift platform which provides a separate server for each individual user. The platform takes care of the provisioning of the server and allocating related to storage.

### First, install and import required libraries, watermark our file, initialise our Spark Session Builder and initialise our environment with required configuration

In [1]:
%pip install watermark
%pip install Minio
%pip install matplotlib


You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/opt/app-root/bin/python3.8 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import json
from pyspark import SparkConf
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import from_json, col, to_json, struct
import watermark
from minio import Minio

%matplotlib inline
%load_ext watermark

In [3]:
%watermark -n -v -m -g -iv


Python implementation: CPython
Python version       : 3.8.6
IPython version      : 7.22.0

Compiler    : GCC 8.4.1 20200928 (Red Hat 8.4.1-1)
OS          : Linux
Release     : 4.18.0-305.34.2.el8_4.x86_64
Machine     : x86_64
Processor   : x86_64
CPU cores   : 16
Architecture: 64bit

Git hash: 9f7107aa0927f093c75a4e37017c6b0c38425c71

watermark: 2.3.0
json     : 2.0.9



## Load parameters

In [4]:
from config import get_params
user_id,PROJECT_NAME,EXPERIMENT_NAME,experiment_name, s3BucketFullPath = get_params()

STUDENT CONFIGURATION
User ID: "user29"
Project name: "CustomerChurn-user29"
Experiment name: "CustomerChurn-user29", "customerchurn-user29"
S3 Bucket full storage path: "full_data_csv-user29/your.csv"


In [5]:
sparkSessionBuilder = SparkSession\
    .builder\
    .appName("Customer Churn ingest Pipeline")

In [6]:
# submit_args = "--conf spark.jars.ivy=/tmp \
# --conf spark.hadoop.fs.s3a.endpoint=http://minio-ml-workshop:9000 \
# --conf spark.hadoop.fs.s3a.access.key=minio \
# --conf spark.hadoop.fs.s3a.secret.key=minio123 \
# --conf spark.hadoop.fs.s3a.path.style.access=true \
# --conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem \
# --packages org.apache.hadoop:hadoop-aws:3.2.0"

submit_args = "--conf spark.jars.ivy=/tmp \
--conf spark.hadoop.fs.s3a.endpoint=http://minio-ml-workshop:9000 \
--conf spark.hadoop.fs.s3a.access.key=minio \
--conf spark.hadoop.fs.s3a.secret.key=minio123 \
--conf spark.hadoop.fs.s3a.path.style.access=true \
--conf spark.hadoop.fs.s3a.impl=org.apache.hadoop.fs.s3a.S3AFileSystem \
--packages org.apache.hadoop:hadoop-aws:3.2.0,\
org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.1,\
org.apache.kafka:kafka-clients:2.8.0,\
org.apache.spark:spark-streaming_2.12:3.0.1"

###  Connect to Spark Cluster provided by OpenShift Platform

In [7]:
import spark_util

spark = spark_util.getOrCreateSparkSession("ML Ops Demo", submit_args)
spark.sparkContext.setLogLevel("INFO")
print('Spark context started.')

Initializing environment variables for Spark
Creating a spark session...
Spark session created


Spark context started.


###  Declare our input data sources, import and combine them

In [8]:
dataFrame_Customer = spark.read\
                .options(delimeter=',', inferSchema='True', header='True') \
                .csv("s3a://rawdata/customers/Customer-Churn_P1.csv")
dataFrame_Customer.printSchema()

root
 |-- customerID: integer (nullable = true)
 |-- gender: string (nullable = true)
 |-- SeniorCitizen: integer (nullable = true)
 |-- Partner: string (nullable = true)
 |-- Dependents: string (nullable = true)
 |-- tenure: integer (nullable = true)



In [9]:
# dataFrame_Products = spark.read\
#                 .options(delimeter=',', inferSchema='True', header='True') \
#                 .csv("s3a://rawdata/products/Customer-Churn_P2.csv")
# dataFrame_Products.printSchema()


In [12]:
from pyspark.sql.types import *
from  pyspark.sql.functions import *

srcKafkaBrokers = "odh-message-bus-kafka-bootstrap:9092"
srcKakaTopic = "datatelco"



schema = StructType()\
    .add("customerID", IntegerType())\
    .add("PhoneService", StringType())\
    .add("MultipleLines", StringType())\
    .add("InternetService", StringType())\
    .add("OnlineSecurity", StringType())\
    .add("OnlineBackup", StringType())\
    .add("DeviceProtection", StringType())\
    .add("TechSupport", StringType())\
    .add("StreamingTV", StringType())\
    .add("StreamingMovies", StringType())\
    .add("Contract", StringType())\
    .add("PaperlessBilling", StringType())\
    .add("PaymentMethod", StringType())\
    .add("MonthlyCharges", StringType())\
    .add("TotalCharges", DoubleType())\
    .add("Churn", StringType())



#Read from JSON Kafka messages into a dataframe
dataFrame_Products = spark.read.format("kafka")\
    .option("kafka.bootstrap.servers", srcKafkaBrokers)\
    .option("subscribe", srcKakaTopic)\
    .option("startingOffsets", "earliest")\
    .load()\
    .withColumn("value", regexp_replace(col("value").cast("string"), "\\\\", "")) \
    .withColumn("value", regexp_replace(col("value"), "^\"|\"$", "")) \
    .selectExpr("CAST(value AS STRING) as jsonValue")\
    .rdd.map(lambda row: row["jsonValue"])

dfObj = spark.read.schema(schema).json(dataFrame_Products)
dfObj.printSchema()
dfObj.show(n=2)


root
 |-- customerID: integer (nullable = true)
 |-- PhoneService: string (nullable = true)
 |-- MultipleLines: string (nullable = true)
 |-- InternetService: string (nullable = true)
 |-- OnlineSecurity: string (nullable = true)
 |-- OnlineBackup: string (nullable = true)
 |-- DeviceProtection: string (nullable = true)
 |-- TechSupport: string (nullable = true)
 |-- StreamingTV: string (nullable = true)
 |-- StreamingMovies: string (nullable = true)
 |-- Contract: string (nullable = true)
 |-- PaperlessBilling: string (nullable = true)
 |-- PaymentMethod: string (nullable = true)
 |-- MonthlyCharges: string (nullable = true)
 |-- TotalCharges: double (nullable = true)
 |-- Churn: string (nullable = true)

+----------+------------+----------------+---------------+--------------+------------+----------------+-----------+-----------+---------------+--------------+----------------+----------------+--------------+------------+-----+
|customerID|PhoneService|   MultipleLines|InternetService

In [14]:
dataFrom_All = dataFrame_Customer.join(dfObj, "customerID", how="full")

###  Push prepared data to object storage and stop Spark cluster to save resources

In [15]:
file_location = "s3a://data/full_data_csv-" + user_id
dataFrom_All.repartition(1).write.mode("overwrite")\
    .option("header", "true")\
    .format("csv").save(file_location)

In [16]:
spark.stop()

In [17]:
print('Notebook complete!')

Notebook complete!
